In [49]:
#########################################################################
#Data Collection 
#
# Functions for handling subwatershed characteristics data collection
#
# Authors: Shannon McAvoy (smcavoy@dewberry.com)
#
# Editor: Evan Barnes (ebarnes@dewberry.com)
#
# Copyright: Dewberry Engineers Inc.
#########################################################################

In [50]:
import pandas as pd
import arcpy
import numpy as np
from numpy import mean
from numpy import std
import os
from arcpy import management
# from arcgis.gis import GIS
from time import time
arcpy.CheckOutExtension('Spatial')
# import rasterio

'CheckedOut'

In [51]:
arcpy.env.overwriteOutput = True

In [52]:
path = "P:\Temp\jMatney\Indiana_ML_2"

In [53]:
st = time()
# Set Local Variables
output_workspace = os.path.join(path, "working_dsn")

arcpy.env.workspace = output_workspace

##### change these based on what subwatersheds you want #####

#set source for shapefile with all 11 subwatersheds in the county

# all_subwatersheds = r"P:\Temp\McAvoy\ML_DataCollection\Indiana_ML\Indiana_HUC12_groups\1.shp"
all_subwatersheds = os.path.join(path, "50sws_10groups\\hotfix_last25.shp")

#set source for streams data
streams = os.path.join(path, "Indiana_50c_streams_edited\Indiana_50c_streams_edited.shp")

#####these are the same for all subwatersheds in Indiana#####

#set source for dem
dem_Indiana = os.path.join(path, "Indiana_3dep\Indiana_dem2")

#set source for slope dem
dem_slope = os.path.join(path, "Indiana_SlopeRaster\IN_slope2")

#set source for NHFL Data
nfhl_sfha = os.path.join(path, "NFHL_18_20200310.gdb\S_FLD_HAZ_AR")

#set source for water bodies data 
water_bodies = os.path.join(path, "IndianaMAP_WaterBodies\Water_Bodies_Lakes_LocalRes\Hydrography_LocalRes_WaterbodyDiscrete_NHD_IN.shp")

#set source for dams data 
dams = os.path.join(path, "IndianaMAP_Dams\Dams_IDNR\Dams_IDNR_IN.shp")

#set source for bridges data
bridges = os.path.join(path, "IndianaMAP_Bridges\Bridges_County_INDOT\Bridges_County_INDOT_IN.shp")

#set source for streets data
streets = os.path.join(path, "IndianaMAP_Streets\Streets_Centerlines_IGIO\County_Street_Centerlines_IGIO_IN.gdb\County_Street_Centerlines_IGIO_IN_Dec2019")

#set source for railraods data
railroads = os.path.join(path, "IndianaMAP_Railroads\Railroads_Active_Abandoned_INDOT\Rail_System_Active_Abandoned_INDOT_IN.shp")

#set source for ACS population data
population = os.path.join(path, "Indiana_PopulationData\Indiana_PopulationData.shp")

#set source for ACS median income data 
median_income = os.path.join(path, "Indiana_IncomeData\Indiana_IncomeData.shp")

#set source for county boundary data
county_boundary = os.path.join(path, "MarionCounty_Boundary\Marion_County_Boundary.shp")

#source for Bing building footprints for Indiana (attrubuted with open street maps data)
building_footprints = os.path.join(path, "BuildingFootprints_Indiana\Building_Footprints_Attributed_IN.shp")

#folder with partial duration files, set as workspace temporarily, then reset when done
directory_rainfall = os.path.join(path, "MarionCounty_Rainfall\All_Rainfall_Clipped_IN")

#set source for nlcd land use data
lu_usa = os.path.join(path, r"NLCD_Impervious\NLCD_indiana_polygon.shp")

#set source for impervious indicator data
impervious_usa = os.path.join(path, "NLCD_Impervious\\NLCD_2016_Impervious_L48_20190405_PERCENT\\NLCD_2016_Impervious_L48_20190405.img")


#########################################################################################################
print(round(((time()-st)/60), 2) , 'minutes to process.')

0.13 minutes to process.


In [54]:
subwatershed_list = []
area_list = []
perimeter_list = []
watershed_length_list = []
elongation_ratio_list = []
shape_factor_list = []
circulatory_ratio_list = []
relief_list = []
relief_ratio_list = []
avg_slope_list = []
drainage_density_list = []
ruggedness_list = []
aae_list = []
buildings_aae_list = []
x_list = []
buildings_x_list = []
water_bodies_list = []
dams_list = []
bridges_list = []
streets_list = []
railroads_list = []
population_list = []
dependent_population_list = []
population_density_list = []
avg_median_income_list = []
housing_density_list = []
population_change_list = []
dist_to_stream_avg_list = []
dist_to_stream_stdev_list = []

lu_21_list = []
lu_22_list = []
lu_23_list = []
lu_24_list = []
lu_41_list = []
lu_82_list = []
impervious_percent_list = []

orb100yr06h_list = []
orb100yr12h_list = []
orb100yr24h_list = []
orb25yr06h_list = []
orb25yr12h_list = []
orb25yr24h_list = []
orb2yr06h_list = []
orb2yr12h_list = []
orb2yr24h_list = []
orb50yr06h_list = []
orb50yr12h_list = []
orb50yr24h_list = []
orb100yr06ha_am_list = []
orb100yr12ha_am_list = []
orb100yr24ha_am_list = []
orb25yr06ha_am_list = []
orb25yr12ha_am_list = []
orb25yr24ha_am_list = []
orb2yr06ha_am_list = []
orb2yr12ha_am_list = []
orb2yr24ha_am_list = []
orb50yr06ha_am_list = []
orb50yr12ha_am_list = []
orb50yr24ha_am_list = []

In [55]:
print(all_subwatersheds)

P:\Temp\jMatney\Indiana_ML_2\50sws_10groups\hotfix_last25.shp


In [56]:
#add AREA and PERIMETER fields for the whole subwatersheds file
#calculate area of subwatersheds
arcpy.AddField_management(all_subwatersheds, "AREA", "DOUBLE")

arcpy.CalculateGeometryAttributes_management(all_subwatersheds, "AREA AREA_GEODESIC", '', 
                                                    "SQUARE_KILOMETERS",
                                                    None)
#calculate perimeter of subwatershed
arcpy.AddField_management(all_subwatersheds, "PERIMETER", "DOUBLE")
perimeter = arcpy.CalculateGeometryAttributes_management(all_subwatersheds, "PERIMETER PERIMETER_LENGTH_GEODESIC",
                                                         "KILOMETERS")

print("perimeter calculated")

perimeter calculated


In [57]:
st = time()
# perform all need intersections here:

#put all intersections in this projection so area and length will be in meters
arcpy.env.outputCoordinateSystem = arcpy.SpatialReference(102008)

water_bodies_intersect = "water_bodies.shp"
water_bodies_intersect = arcpy.Intersect_analysis([water_bodies, all_subwatersheds], water_bodies_intersect)

population_intersect = "population.shp"
arcpy.Intersect_analysis([population, all_subwatersheds], population_intersect)

median_income_intersect= "median_income.shp"
arcpy.Intersect_analysis([median_income, all_subwatersheds], median_income_intersect)

building_footprints_intersect = "building_footprints.shp"
arcpy.Intersect_analysis([building_footprints, all_subwatersheds], building_footprints_intersect)

streams_intersect = "streams.shp"
arcpy.Intersect_analysis([streams, all_subwatersheds], streams_intersect)


print(round(((time()-st)/60), 2) , 'minutes to process.')

20.44 minutes to process.


In [58]:
def get_length_area(layer, length_area):
    #length_Area = 'LENGTH' or 'AREA'
    search_field = "SHAPE@{}".format(length_area)
    values = []
    with arcpy.da.SearchCursor(layer, [search_field]) as sCursor: 
        for s in sCursor:
            # convert meters to kilometers
            if length_area == 'AREA':
                value = s[0]/ (1000 * 1000)
            else: # assumed length
                value = s[0] / 1000
                    
            values.append(value)
    return values

In [59]:
st = time()
# make all layers for selections
#subwatershed_selection = "subwatershed_lyr"
#arcpy.MakeFeatureLayer_management (all_subwatersheds, subwatershed_selection)

# nfhl_sfha_selection = "nfhl_sfha_lyr"
# arcpy.MakeFeatureLayer_management(nfhl_sfha_intersect, nfhl_sfha_selection)

water_bodies_selection = "water_bodies_lyr"
arcpy.MakeFeatureLayer_management(water_bodies_intersect, water_bodies_selection)

# dams_selection = "dams_lyr"
# arcpy.MakeFeatureLayer_management(dams_intersect, dams_selection)

# bridges_selection = "bridges_lyr"
# arcpy.MakeFeatureLayer_management(bridges_intersect, bridges_selection)

# streets_selection = "streets_lyr"
# arcpy.MakeFeatureLayer_management(streets_intersect, streets_selection)

# railroads_selection = "railroads_lyr"
# arcpy.MakeFeatureLayer_management(railroads_intersect, railroads_selection)

population_selection = "population_lyr"
arcpy.MakeFeatureLayer_management(population_intersect, population_selection)

median_income_selection = "median_income_lyr"
arcpy.MakeFeatureLayer_management(median_income_intersect, median_income_selection)

building_footprints_selection= "building_footprints_lyr"
arcpy.MakeFeatureLayer_management(building_footprints_intersect, building_footprints_selection)

streams_selection = "streams_lyr"
arcpy.MakeFeatureLayer_management(streams_intersect, streams_selection)

# lu_selection = "lu_lyr"
# arcpy.MakeFeatureLayer_management(lu_subwatershed, lu_selection)

print(round(((time()-st)/60), 2) , 'minutes to process.')

5.05 minutes to process.


In [60]:
st = time()


#search cursor through each row of county subwatersheds file
with arcpy.da.SearchCursor(all_subwatersheds, ['HUC12','AREA', 'PERIMETER']) as cursor:

    for row in cursor:
        subwatershed_number = row[0]
        
        subwatershed_list.append(subwatershed_number)

        print("--------------------", subwatershed_number, " is the subwatershed to be worked on ------------------------")
        # Make a layer from the feature class
        arcpy.MakeFeatureLayer_management(all_subwatersheds,"subwatersheds_lyr")
        
        #select subwatershed from shapefile with all of them
        s = arcpy.SelectLayerByAttribute_management(all_subwatersheds, "NEW_SELECTION", 
                                                "HUC12 = '{}'".format(subwatershed_number))
        arcpy.CopyFeatures_management(s, "s_lyr.shp")
        subwatershed_selection1 = os.path.join(path, "working_dsn\s_lyr.shp")
        #arcpy.MakeFeatureLayer_management(s, subwatershed_selection1)
        
        # use mask in lieu of clipping rasters
        arcpy.env.cellSize = dem_slope
        arcpy.env.snapRaster = dem_slope
        #arcpy.env.outputCoordinateSystem = 
        arcpy.env.mask = subwatershed_selection1

        area = row[1]
        area_list.append(area)
        print(area, " square kilometers")

        perimeter = row[2]
        perimeter_list.append(perimeter)
        print(perimeter, " kilometers")


        #calculate area covered by lakes/reserviors 
        arcpy.management.SelectLayerByAttribute(water_bodies_selection, "NEW_SELECTION", 
                                                            "HUC12 = '{}'".format(subwatershed_number))
        water_bodies_area_sum = sum(get_length_area(water_bodies_selection, "AREA"))
        water_bodies_list.append(water_bodies_area_sum)
        print(water_bodies_area_sum, " = area of all water bodies in the subwatershed (square km)")


        #all streams in subwatershed
        arcpy.management.SelectLayerByAttribute(streams_selection, "NEW_SELECTION", 
                                                            "HUC12 = '{}'".format(subwatershed_number))
        print("streams clipped")
        print("length calculated")
        #get the sum of all of the areas
        stream_length_list = get_length_area(streams_selection, "LENGTH")  
        stream_length_sum = sum(stream_length_list)
        print(stream_length_sum, " = length of all streams in subwatershed")
        print(area, " = subwatershed area")
        drainage_density = stream_length_sum / area
        drainage_density_list.append(drainage_density)
        print(drainage_density, "is the drainage density (streams/km)")
        #find watershed length
        #watershed length = distance from outlet to watershed boundary along the main channel
        #we are assuming that the longest stream above is the main channel
        watershed_length = max(stream_length_list)
        watershed_length_list.append(watershed_length)
        print(watershed_length, " = watershed length (longest stream in subwatershed)")
        #calculate shape factor 
        #watershed length squared divided by watershed area
        shape_factor = (watershed_length**2) / area
        shape_factor_list.append(shape_factor)
        print(shape_factor, " is the shape factor.")

        
        #use Raster Domain tool to get z-enabled polyline of perimeter
        dem_clip = os.path.join(path, "working_dsn\dem_clip.tif")
        arcpy.Clip_management(dem_Indiana, "#", dem_clip, subwatershed_selection1, "#", "ClippingGeometry", "NO_MAINTAIN_EXTENT")
        perimeter_polyline = "perimeter_polyline.shp"
        arcpy.RasterDomain_3d(dem_clip, perimeter_polyline, "LINE")
        print("created 3d polyline of subwatershed perimeter")

        #Convert each vertices of the polyline into points
        perimeter_points = "perimeter_points.shp"
        arcpy.FeatureVerticesToPoints_management(perimeter_polyline, perimeter_points)
        print("created perimeter points")

        #get Z values into the attribute table for the points
        arcpy.AddZInformation_3d(perimeter_points, "Z")
        print("added z information")

        #get the max value from all of the points
        perimeter_stats = "perimeter_stats"
        arcpy.Statistics_analysis(perimeter_points, "perimeter_stats", [["Z", "MAX"], ["Z", "MIN"]])

        #get the values for max Z and min Z into a format to use them
        with arcpy.da.SearchCursor(perimeter_stats, ['MAX_Z', 'MIN_Z']) as statsCursor:
            for stat in statsCursor:
                max_z = stat[0]
                min_z = stat[1]


        print(max_z)
        print(min_z)

        relief = max_z - min_z
        
        #calculate relief ratio
        #relief divided by watershed length
        #length is is kilometers, convert to meters
        watershed_length_meters = watershed_length * 1000
        relief_ratio = relief / watershed_length_meters
        relief_ratio_list.append(relief_ratio)
        print(relief_ratio, " = relief ratio")


        #calculate ruggedness number
        #product of relief and drainage density
        #relief is in meters, convert first to km
        relief_km = relief / 1000
        print(relief_km, " is the relief in km")
        ruggedness = relief_km * drainage_density
        ruggedness_list.append(ruggedness)
        print(ruggedness, " is the ruggedness number")
        
        
                #ACS population data - 5 year estimates (2014-2018) gotten from ESRI Living Atlas Data
        #data is chosen to be on census tract level
        arcpy.management.SelectLayerByAttribute(population_selection, "NEW_SELECTION", 
                                                            "HUC12 = '{}'".format(subwatershed_number))
        #get total population
        #field = B01001_001E = Total Population (alias)
        #when clipped, field name changes to B01001_001

        #get the sum of all of the populations in each tract
        total_pop_list = []  
        with arcpy.da.SearchCursor(population_selection, ['B01001_001']) as popCursor:  
            for pop in popCursor:  
                total_pop_value = pop[0] 
                total_pop_list.append(total_pop_value)  
        total_pop_sum = sum(total_pop_list)
        population_list.append(total_pop_sum)
        print(total_pop_sum, " = total population in the subwatershed") 

        #get the average of percentages of dependent age groups in each tract
        #field = B01001_calc_pctDependE = Percent of Population in Dependent Age Groups (under 18 and 65+) (alias)
        #when clipped field changes to B01001_61
        dependent_pop_list = []  
        with arcpy.da.SearchCursor(population_selection, ["B01001__61"]) as dpopCursor: 
            for dpop in dpopCursor:  
                dependent_pop_value = dpop[0] 
                dependent_pop_list.append(dependent_pop_value)  
        dependent_pop_avg_pct = mean(dependent_pop_list)
        dependent_population_list.append(dependent_pop_avg_pct)

        print(dependent_pop_avg_pct, " = total percent of dependent population in the subwatershed") 

        #find population density
        population_density = total_pop_sum / area
        population_density_list.append(population_density)
        print(population_density, " = population density of subwatershed (people/square km)")
        

        #get housing density
        #select buildings that are marked residential
        arcpy.SelectLayerByAttribute_management(building_footprints_selection, "NEW_SELECTION",
                                                                             "HUC12 = '{}' And RES_NONRES = 'Res'".format(subwatershed_number))
        #get count of how many buildings there are 
        buildings_count = arcpy.GetCount_management(building_footprints_selection)
        buildings_count_number = buildings_count.getOutput(0)
        print(buildings_count_number, " = number of residential building footprints in the subwatershed")
        #divide number of buildings by subwatershed area
        housing_density = int(buildings_count_number) / area
        housing_density_list.append(housing_density)
        print(housing_density, " = housing density (buildings per square km)")

    

    #########################################################################################




print(round(((time()-st)/60), 2) , 'minutes to process.')    


-------------------- 051402010902  is the subwatershed to be worked on ------------------------
53.5515601333  square kilometers
40.2372421039  kilometers
1.169741078052342  = area of all water bodies in the subwatershed (square km)
streams clipped
length calculated
12.147988121800289  = length of all streams in subwatershed
53.5515601333  = subwatershed area
0.22684657723438195 is the drainage density (streams/km)
12.147988121800289  = watershed length (longest stream in subwatershed)
2.7557295257143237  is the shape factor.
created 3d polyline of subwatershed perimeter
created perimeter points
added z information
191.267974853
118.535255432
0.005987223455584122  = relief ratio
0.072732719421  is the relief in km
0.016499168453602508  is the ruggedness number
11085  = total population in the subwatershed
40.75  = total percent of dependent population in the subwatershed
206.9967704471603  = population density of subwatershed (people/square km)
812  = number of residential building foo

created 3d polyline of subwatershed perimeter
created perimeter points
added z information
276.496612549
175.08114624
0.004122357614189643  = relief ratio
0.10141546630900002  is the relief in km
0.03804761412554601  is the ruggedness number
9157  = total population in the subwatershed
39.2  = total percent of dependent population in the subwatershed
139.6425988979935  = population density of subwatershed (people/square km)
94  = number of residential building footprints in the subwatershed
1.4334830508257497  = housing density (buildings per square km)
-------------------- 051201011404  is the subwatershed to be worked on ------------------------
41.8061956966  square kilometers
31.8306441238  kilometers
0.09285627206926342  = area of all water bodies in the subwatershed (square km)
streams clipped
length calculated
19.857235940889026  = length of all streams in subwatershed
41.8061956966  = subwatershed area
0.47498308827234353 is the drainage density (streams/km)
10.843056515016963 

51.1699930921  square kilometers
40.6817133062  kilometers
6.5765106903062724  = area of all water bodies in the subwatershed (square km)
streams clipped
length calculated
24.97560256063497  = length of all streams in subwatershed
51.1699930921  = subwatershed area
0.48809079406522116 is the drainage density (streams/km)
10.173067693665349  = watershed length (longest stream in subwatershed)
2.022499907585005  is the shape factor.
created 3d polyline of subwatershed perimeter
created perimeter points
added z information
237.958999634
210.354598999
0.0027134785166316113  = relief ratio
0.02760440063500002  is the relief in km
0.013473453825631655  is the ruggedness number
8860  = total population in the subwatershed
41.6  = total percent of dependent population in the subwatershed
173.1483524739399  = population density of subwatershed (people/square km)
785  = number of residential building footprints in the subwatershed
15.341022200004831  = housing density (buildings per square km)
-

28674  = total population in the subwatershed
40.13333333333333  = total percent of dependent population in the subwatershed
380.6983921369991  = population density of subwatershed (people/square km)
801  = number of residential building footprints in the subwatershed
10.634700847518179  = housing density (buildings per square km)
-------------------- 051202030405  is the subwatershed to be worked on ------------------------
99.150668903  square kilometers
53.1513407023  kilometers
0.7900594043778376  = area of all water bodies in the subwatershed (square km)
streams clipped
length calculated
54.36340845839042  = length of all streams in subwatershed
99.150668903  = subwatershed area
0.5482908896113917 is the drainage density (streams/km)
23.502285586149824  = watershed length (longest stream in subwatershed)
5.570889575271773  is the shape factor.
created 3d polyline of subwatershed perimeter
created perimeter points
added z information
263.933135986
176.796539307
0.003707579688775064

In [62]:
st = time()
outputs = {'subwatershed': subwatershed_list,
          'area': area_list,
          'perimeter': perimeter_list,
           'watershed_length': watershed_length_list,
           'drainage_density': drainage_density_list,
           'shape_factor': shape_factor_list,
           'ruggedness': ruggedness_list,
           'population_density': population_density_list,
        'housing_density': housing_density_list

          }

In [61]:
len(watershed_length_list)

24

In [63]:
            
outputs_df = pd.DataFrame(outputs, columns = ['subwatershed',
                                             'area',
                                             'perimeter',
                                              'watershed_length',
                                              'drainage_density',
                                              'shape_factor',

                                              'ruggedness',
  
                                              'population_density',
                                              'housing_density'
                                             ])    
    

print(outputs_df)

outputs_df.to_excel(os.path.join(path,"working_dsn\\hotfix_55_79.xlsx"))
print(round(((time()-st)/60), 2) , 'minutes to process.')

    subwatershed        area  perimeter  watershed_length  drainage_density  \
0   051402010902   53.551560  40.237242         12.147988          0.226847   
1   051201060303   59.355492  57.319092         13.885543          0.542773   
2   071200011306   81.091258  59.963772         20.077706          0.351442   
3   051202060105  130.966552  59.880817         28.893195          0.220615   
4   051201011507   94.722519  53.098641         18.845989          0.610711   
5   051202080401   58.804630  42.324744         11.189745          0.505556   
6   051201040101   54.354774  45.811716         13.205990          0.464162   
7   051201100605   65.574546  51.552098         24.601327          0.375166   
8   051201011404   41.806196  31.830644         10.843057          0.474983   
9   051202040404   82.156765  55.312017         10.503399          0.547241   
10  051201081606   66.584437  42.751504         20.454115          0.307191   
11  051202070703   60.308449  41.392003         18.5

In [40]:
len(outputs['population_density'])

54